# 멋진 챗봇 만들기 프로젝트 (트랜스포머 구조, 동의어 데이터 증강, BELU 스코어 사용해보기)

데이터 증강 모델 가져올 때, 구버전 사용하기 (by.승환님)
https://iambeginnerdeveloper.tistory.com/41

## 0. 라이브러리

In [195]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import gensim
import re
import os
import random
import math

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

print(tf.__version__)

2.6.0


## 1. 데이터 다운로드

In [196]:
import pandas as pd

# 데이터 경로와 파일 이름
file_path = '~/aiffel/Chapter18/ChatbotData.csv'

# CSV 파일을 읽어오기
data = pd.read_csv(file_path)

# questions와 answers 컬럼을 각각 리스트로 저장
questions = data['Q'].tolist()
answers = data['A'].tolist()

print("Sample Question:", questions[0])
print("Sample Answer:", answers[0])


Sample Question: 12시 땡!
Sample Answer: 하루가 또 가네요.


## 2. 데이터 정제

In [197]:
def preprocess_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r"[^a-zA-Z0-9가-힣\s.,!?]", "", sentence)
    return sentence

sample_sentence = "Hello, AIFFEL! Let's learn about NLP. #AI #2023"
processed_sentence = preprocess_sentence(sample_sentence)
print("원문:", sample_sentence)
print("전처리 후:", processed_sentence)

원문: Hello, AIFFEL! Let's learn about NLP. #AI #2023
전처리 후: hello, aiffel! lets learn about nlp. ai 2023


## 3. 데이터 토큰화

1. 소스 문장 데이터와 타겟 문장 데이터를 입력으로 받습니다.
2. 데이터를 앞서 정의한 preprocess_sentence() 함수로 정제하고, 토큰화합니다.
3. 토큰화는 전달받은 토크나이즈 함수를 사용합니다. 이번엔 mecab.morphs 함수를 전달하시면 됩니다.
4. 토큰의 개수가 일정 길이 이상인 문장은 데이터에서 제외합니다.
5. 중복되는 문장은 데이터에서 제외합니다. 소스 : 타겟 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사합니다. 중복 쌍이 흐트러지지 않도록 유의하세요!

구현한 함수를 활용하여 questions 와 answers 를 각각 que_corpus , ans_corpus 에 토큰화하여 저장합니다.

In [198]:
import numpy as np
from konlpy.tag import Mecab
mecab = Mecab()

def preprocess_sentence(sentence):
    # 예시 정제 함수 (구체적인 구현은 필요시 추가)
    return sentence.lower()  # 간단히 소문자로 변환한다고 가정

def build_corpus(sentences, tokenizer, max_len=20):
    """
    주어진 문장 데이터를 정제하고 토큰화하여 길이와 중복 조건을 만족하는 코퍼스를 생성합니다.
    
    Parameters:
    - sentences (list of str): 입력 문장 데이터 리스트 (예: questions, answers)
    - tokenizer (function): 토큰화 함수 (예: mecab.morphs)
    - max_len (int): 허용하는 최대 토큰 개수 (기본값: 20)
    
    Returns:
    - list of list of str: 정제 및 토큰화된 코퍼스 데이터
    - list of int: 각 문장의 토큰 길이
    """
    corpus = []
    lengths = []  # 각 문장의 토큰 길이 저장용
    seen_sentences = set()  # 중복 제거를 위한 집합

    for sentence in sentences:
        # 1. 문장 정제
        clean_sentence = preprocess_sentence(sentence)
        
        # 2. 문장 토큰화
        tokenized_sentence = tokenizer(clean_sentence)
        
        # 3. 최대 길이 필터링
        if len(tokenized_sentence) > max_len:
            continue
        
        # 4. 중복 문장 제거
        tokenized_str = ' '.join(tokenized_sentence)
        if tokenized_str not in seen_sentences:
            seen_sentences.add(tokenized_str)
            corpus.append(tokenized_sentence)
            lengths.append(len(tokenized_sentence))
    
    return corpus, lengths

que_corpus, que_lengths = build_corpus(questions, mecab.morphs)
ans_corpus, ans_lengths = build_corpus(answers, mecab.morphs)

# 각 열의 평균 길이 계산
que_avg_len = np.mean(que_lengths)
ans_avg_len = np.mean(ans_lengths)

print("Questions Average Length:", que_avg_len)
print("Answers Average Length:", ans_avg_len)

# ans_corpus에서 5개의 예시 출력
print("Examples from que_corpus:")
for i, example in enumerate(que_corpus[:5]):
    print(f"{i + 1}: {example}")
print("Examples from ans_corpus:")
for i, example in enumerate(ans_corpus[:5]):
    print(f"{i + 1}: {example}")


Questions Average Length: 6.992157876594278
Answers Average Length: 8.530188186095138
Examples from que_corpus:
1: ['12', '시', '땡', '!']
2: ['1', '지망', '학교', '떨어졌', '어']
3: ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다']
4: ['3', '박', '4', '일', '정도', '놀', '러', '가', '고', '싶', '다']
5: ['ppl', '심하', '네']
Examples from ans_corpus:
1: ['하루', '가', '또', '가', '네요', '.']
2: ['위로', '해', '드립니다', '.']
3: ['여행', '은', '언제나', '좋', '죠', '.']
4: ['눈살', '이', '찌푸려', '지', '죠', '.']
5: ['다시', '새로', '사', '는', '게', '마음', '편해요', '.']


## 4. 데이터 증강

In [199]:
!pip install --upgrade gensim==3.8.3

In [93]:
from gensim.models import KeyedVectors

# Word2Vec 모델 로드 (ko.bin 파일이 위치한 경로로 변경)
model_path = '~/aiffel/ko.bin'  # 업로드한 파일 경로로 지정
wv = KeyedVectors.load(model_path)
print("모델 로드 완료!")

AttributeError: Can't get attribute 'Vocab' on <module 'gensim.models.word2vec' from '/opt/conda/lib/python3.9/site-packages/gensim/models/word2vec.py'>

In [64]:
from gensim.models import Word2Vec

# ko.bin 파일 경로 설정
model_path = '~/aiffel/ko.bin'

# Word2Vec 모델 로드
wv = Word2Vec.load(model_path)
print("모델 로드 완료!")

# 모델 테스트 예시
print(wv.most_similar("학교"))

AttributeError: Can't get attribute 'Vocab' on <module 'gensim.models.word2vec' from '/opt/conda/lib/python3.9/site-packages/gensim/models/word2vec.py'>

In [65]:
import random

def lexical_sub(sentence, wv, substitution_prob=0.3):
    """
    입력 문장을 Embedding 유사도를 기반으로 일부 단어를 대체하여 데이터 증강을 수행합니다.
    
    Parameters:
    - sentence (str): 입력 문장
    - wv (KeyedVectors): 사전 훈련된 Word2Vec 모델
    - substitution_prob (float): 각 단어를 대체할 확률 (기본값: 0.3)
    
    Returns:
    - str: 대체된 문장
    """
    tokens = sentence.split()
    result = []

    for tok in tokens:
        # substitution_prob에 따라 단어를 유사한 단어로 대체
        if random.random() < substitution_prob:
            try:
                # 가장 유사한 단어를 찾고 치환
                similar_word = wv.most_similar(tok)[0][0]
                result.append(similar_word)
            except KeyError:
                # 모델에 단어가 없으면 원래 단어 그대로 사용
                result.append(tok)
        else:
            # 대체하지 않는 경우 원래 단어 그대로 사용
            result.append(tok)
    
    return ' '.join(result)


In [ ]:
from tqdm import tqdm

# Augmentation된 데이터 저장할 리스트
augmented_que_corpus = []
augmented_ans_corpus = []

# que_corpus의 Augmentation과 ans_corpus의 원본을 병렬 데이터로 생성
for sentence in tqdm(que_corpus):
    augmented_sentence = lexical_sub(' '.join(sentence), wv)
    augmented_que_corpus.append(augmented_sentence.split())  # 토큰화된 형태로 저장

# ans_corpus의 Augmentation과 que_corpus의 원본을 병렬 데이터로 생성
for sentence in tqdm(ans_corpus):
    augmented_sentence = lexical_sub(' '.join(sentence), wv)
    augmented_ans_corpus.append(augmented_sentence.split())  # 토큰화된 형태로 저장

# 최종 데이터를 합쳐서 3배로 확장
final_que_corpus = que_corpus + augmented_que_corpus
final_ans_corpus = ans_corpus + ans_corpus  # que_corpus의 Augmentation과 원본 ans_corpus의 병렬 데이터
final_que_corpus += que_corpus  # ans_corpus의 Augmentation과 원본 que_corpus의 병렬 데이터
final_ans_corpus += augmented_ans_corpus

print("Original corpus size:", len(que_corpus))
print("Augmented corpus size:", len(final_que_corpus))


In [200]:
# ans_corpus에 <start>와 <end> 토큰 추가
ans_corpus = [["<start>"] + sentence + ["<end>"] for sentence in ans_corpus]

# 예시 출력 (첫 3개 문장 확인)
print("First 3 examples from ans_corpus with start/end tokens:")
for i in range(3):
    print(ans_corpus[i])


First 3 examples from ans_corpus with start/end tokens:
['<start>', '하루', '가', '또', '가', '네요', '.', '<end>']
['<start>', '위로', '해', '드립니다', '.', '<end>']
['<start>', '여행', '은', '언제나', '좋', '죠', '.', '<end>']


In [201]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 질문과 답변 데이터를 결합하여 단어 사전 구축
total_corpus = que_corpus + ans_corpus

# Tokenizer 생성 (필요시 num_words로 어휘 크기 제한)
tokenizer = Tokenizer(filters='', oov_token='<unk>')
tokenizer.fit_on_texts([' '.join(sentence) for sentence in total_corpus])

# 단어 사전 정보 출력
print(f"단어 사전에 포함된 단어 수: {len(tokenizer.word_index)}")

# 단어 사전 크기 확인
vocab_size = len(tokenizer.word_index) + 1  # 패딩 토큰 포함


단어 사전에 포함된 단어 수: 6822


In [202]:
# 질문 데이터(que_corpus)를 시퀀스로 변환
enc_train = tokenizer.texts_to_sequences([' '.join(sentence) for sentence in que_corpus])

# 답변 데이터(ans_corpus)를 시퀀스로 변환
dec_train = tokenizer.texts_to_sequences([' '.join(sentence) for sentence in ans_corpus])

# 벡터화된 데이터 예시 출력
print("Sample encoded question:", enc_train[0])
print("Sample encoded answer:", dec_train[0])


Sample encoded question: [2054, 229, 3004, 106]
Sample encoded answer: [3, 241, 8, 132, 8, 47, 2, 4]


In [203]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 패딩 처리 (필요시 maxlen으로 최대 길이 지정)
enc_train = pad_sequences(enc_train, padding='post')
dec_train = pad_sequences(dec_train, padding='post')

print("Shape of enc_train:", enc_train.shape)
print("Shape of dec_train:", dec_train.shape)


Shape of enc_train: (11604, 20)
Shape of dec_train: (7652, 22)


## 오류
차원이 안맞는 오류가 발생하여 dec 크기에 맞춰 enc도 잘라주었음.

In [204]:
# 두 데이터의 최소 길이에 맞춰 슬라이싱
min_len = min(len(enc_train), len(dec_train))
enc_train = enc_train[:min_len]
dec_train = dec_train[:min_len]

print("Updated Shape of enc_train:", enc_train.shape)
print("Updated Shape of dec_train:", dec_train.shape)


Updated Shape of enc_train: (7652, 20)
Updated Shape of dec_train: (7652, 22)


In [205]:
import tensorflow as tf

# Dataset 생성 및 셔플링, 배치 처리
BUFFER_SIZE = 20000
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [206]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

In [207]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [208]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights

In [209]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out

In [210]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

In [211]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        '''
        Masked Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        '''
        Multi-Head Attention
        '''
        residual = out
        out = self.norm_2(out)
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [212]:
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

In [213]:
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [214]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

In [215]:
transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=vocab_size,
    tgt_vocab_size=vocab_size,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)
		
d_model = 512

In [216]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [217]:
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [218]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [219]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [220]:
# 번역 생성 함수
def evaluate(sentence, model, src_tokenizer, tgt_tokenizer, max_len=50):
    sentence = preprocess_sentence(sentence)  # 전처리 함수 사용
    tokens = src_tokenizer.texts_to_sequences([sentence])[0]  # 토큰화

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens], maxlen=max_len, padding='post')

    output = tf.expand_dims([tgt_tokenizer.word_index['<start>']], 0)
    ids = []

    for i in range(max_len):
        predictions, _, _, _ = model(_input, output, None, None, None)
        predicted_id = tf.argmax(predictions[0, -1]).numpy()

        if predicted_id == tgt_tokenizer.word_index['<end>']:
            break
        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    return ' '.join([tgt_tokenizer.index_word[id] for id in ids])

훈련하기

In [221]:
import tensorflow as tf
from tqdm import tqdm

# 하이퍼파라미터 설정
EPOCHS = 8
BATCH_SIZE = 64
BUFFER_SIZE = len(questions)
vocab_size = len(tokenizer.word_index) + 1  # 단어 사전 크기

# 1. Dataset 생성
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# 모델 및 옵티마이저 초기화
transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=vocab_size,
    tgt_vocab_size=vocab_size,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True
)
learning_rate = LearningRateScheduler(512, warmup_steps=1000)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# 2. 훈련 루프
for epoch in range(EPOCHS):
    total_loss = 0
    dataset_count = tf.data.experimental.cardinality(dataset).numpy()
    tqdm_bar = tqdm(total=dataset_count, desc=f'Epoch {epoch + 1}', unit='batch')

    for (batch, (src, tgt)) in enumerate(dataset):
        loss, enc_attns, dec_attns, dec_enc_attns = train_step(src, tgt, transformer, optimizer)
        total_loss += loss
        tqdm_bar.set_postfix({'Loss': total_loss.numpy() / (batch + 1)})
        tqdm_bar.update(1)
    
    tqdm_bar.close()
    print(f'Epoch {epoch + 1} Loss: {total_loss.numpy() / dataset_count:.4f}')

# 예문 번역 생성
test_questions = [
    "지루하다, 놀러가고 싶어.",
    "오늘 일찍 일어났더니 피곤하다.",
    "간만에 여자친구랑 데이트 하기로 했어.",
    "집에 있는다는 소리야."
]

for question in test_questions:
    print(f"\n질문: {question}")
    predicted_answer = evaluate(question, transformer, tokenizer, tokenizer)
    print(f"예측 답변: {predicted_answer}")


Epoch 1: 100%|██████████| 119/119 [00:10<00:00, 11.49batch/s, Loss=5.77]


Epoch 1 Loss: 5.7688


Epoch 2: 100%|██████████| 119/119 [00:06<00:00, 17.61batch/s, Loss=3.83]


Epoch 2 Loss: 3.8294


Epoch 3: 100%|██████████| 119/119 [00:06<00:00, 17.54batch/s, Loss=3.03]


...

Epoch 6 Loss: 1.2901


Epoch 7: 100%|██████████| 119/119 [00:06<00:00, 17.32batch/s, Loss=1.23]


Epoch 7 Loss: 1.2322


Epoch 8: 100%|██████████| 119/119 [00:06<00:00, 17.37batch/s, Loss=1.27]


Epoch 8 Loss: 1.2678

질문: 지루하다, 놀러가고 싶어.
예측 답변: 사람 마다 다르 지 않 는 게 좋 아요 .

질문: 오늘 일찍 일어났더니 피곤하다.
예측 답변: 천천히 그만두 세요 .

질문: 간만에 여자친구랑 데이트 하기로 했어.
예측 답변: 나쁜 마음 이 없 어서 좋 겠 어요 .

질문: 집에 있는다는 소리야.
예측 답변: 사람 마다 다르 지 않 는 게 좋 아요 .


In [222]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np

def calculate_bleu(reference, candidate):
    """
    BLEU 점수를 계산합니다.
    
    Parameters:
    - reference (list of str): 실제 정답 문장 (리스트 형태로)
    - candidate (list of str): 모델이 예측한 답변 (리스트 형태로)
    
    Returns:
    - float: 계산된 BLEU 점수
    """
    # BLEU 계산 시, smoothing function을 사용해 0 점수를 방지합니다.
    smoothie = SmoothingFunction().method4
    score = sentence_bleu([reference], candidate, smoothing_function=smoothie)
    return score


In [223]:
# 예시 질문과 모델 예측 답변, 실제 답변 (여기서는 예시를 사용)
sample_question = "오늘 날씨 어때?"
sample_reference = ["오늘", "날씨", "맑아요"]
sample_candidate = ["오늘", "날씨", "좋아요"]  # 모델의 예측 결과 예시

# BLEU 점수 계산
bleu_score = calculate_bleu(sample_reference, sample_candidate)
print(f"BLEU Score: {bleu_score:.4f}")


BLEU Score: 0.2118


In [227]:
# 예시 질문과 모델 예측 답변, 실제 답변 (여기서는 예시를 사용)
sample_question = "오늘 일찍 일어났더니 피곤하다."
sample_reference = ["오늘", "일찍", "주무세요"]
sample_candidate = ["천천히", "그만두", "세요"]  # 모델의 예측 결과 예시

# BLEU 점수 계산
bleu_score = calculate_bleu(sample_reference, sample_candidate)
print(f"BLEU Score: {bleu_score:.4f}")

BLEU Score: 0.0000


회고

트랜스포머의 구조와 학습속도, 손실함수를 한번 더 톺아볼 수 있었다.
데이터 증강에서 버전을 다운그레이드 했는데도 해결이 안되어서 궁금하다.
챗봇의 경우, 번역보다는 belu 스코어를 활용하기 어려운 것 같다.